In [2]:
import os 
import speech_recognition as sr 
from googletrans import Translator
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from flask import Flask, request, jsonify
from flask_cors import CORS
from pydub import AudioSegment  
from transformers import pipeline
import spacy
import moviepy.editor as mp 
from moviepy.editor import VideoFileClip

app = Flask(__name__)
CORS(app, origins="http://localhost:3000", supports_credentials=True)

nlp = spacy.load("en_core_web_sm")

@app.route('/api/analyze-audio/<string:text>', methods=['GET'])
def analyze_text(text):
    doc = nlp(text) 
    person_count = len([ent.text for ent in doc.ents if ent.label_ == "PERSON"]) 
    topics = [token.text for token in doc if token.is_alpha and not token.is_stop]
    
    return jsonify({'person_count' : person_count + 1 , 'topic' : topics[0], 'topic2' : topics[1]})
 

@app.route('/api/convert-video-to-mp3', methods=['POST'])
def convert_video_to_mp3():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    video_file = request.files['file']
    if video_file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if video_file: 
        try: 
            video_path = os.path.join('./uploads', video_file.filename)  
            print(video_path)
            video_file.save(video_path)
 
            audio_path = os.path.abspath('./uploads/output.mp3') 
            video_clip = VideoFileClip(video_path)
            audio_clip = video_clip.audio
            audio_clip.write_audiofile(audio_path)
            audio_clip.close()
            video_clip.close()

            return jsonify({'message': 'Video converted to MP3 successfully', 'audio_file': audio_path}), 200
        except Exception as e:
            return jsonify({'error': 'Conversion error', 'details': str(e)}), 500
        
def split_audio(input_audio, output_dir, segment_length_ms=60000):
    print("Proccessing Input........") 
    audio = AudioSegment.from_file(input_audio) 
    total_length_ms = len(audio) 
    num_segments = total_length_ms // segment_length_ms 

    for i in range(num_segments):
        start_time = i * segment_length_ms
        end_time = (i + 1) * segment_length_ms
        segment = audio[start_time:end_time] 
        segment.export(os.path.join(output_dir, f"segment_{i}.wav"), format="wav")
        
    if total_length_ms % segment_length_ms > 0:
        start_time = num_segments * segment_length_ms
        end_time = total_length_ms
        last_segment = audio[start_time:end_time]
        last_segment.export(os.path.join(output_dir, f"segment_{num_segments}.wav"), format="wav")

def transcribe_audio_segments(segment_dir):
    print("Converting to text........")
    recognizer = sr.Recognizer()
    
    transcribed_texts = []
    
    for filename in os.listdir(segment_dir):
        if filename.endswith(".wav"):
            audio_file = os.path.join(segment_dir, filename)
            
            with sr.AudioFile(audio_file) as source:
                audio_text = recognizer.record(source)
                text = recognizer.recognize_google(audio_text)
                transcribed_texts.append(text)
    
    return transcribed_texts

def join_transcribed_texts(texts):
    return " ".join(texts)


@app.route('/api/convert-mp3-to-text', methods=['POST'])
def convert_mp3_to_text():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:
        mp3_filename = os.path.join('uploads', file.filename)
        file.save(mp3_filename)
        segment_dir = 'segments'  
        os.makedirs(segment_dir, exist_ok=True)
        split_audio(mp3_filename, segment_dir, segment_length_ms=60000)
        transcribed_texts = transcribe_audio_segments(segment_dir) 
        joined_text = join_transcribed_texts(transcribed_texts)

        return jsonify({'text': joined_text})

        

@app.route('/api/convert-mp4-to-text', methods=['POST'])
def convert_mp4_to_text():
    file = os.path.abspath('./uploads/output.mp3') 

    if file: 
        mp3_filename = file
        wav_filename = mp3_filename.replace('.mp3', '.wav') 
        AudioSegment.from_mp3(mp3_filename).export(wav_filename, format="wav")

    segment_dir = 'segments'
    try:
        if not os.path.exists(segment_dir):
            os.mkdir(segment_dir)
    except Exception as e:
        print(f"Error creating directory: {e}")

    split_audio(wav_filename, segment_dir, segment_length_ms=60000)  
    transcribed_texts = transcribe_audio_segments(segment_dir) 
    joined_text = join_transcribed_texts(transcribed_texts)
    return jsonify({'text': joined_text})
        
if __name__ == '__main__': 
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


./uploads\final.mp4
MoviePy - Writing audio in c:\Audio_Analysis_UI_UX\uploads\output.mp3


127.0.0.1 - - [14/Nov/2023 22:14:07] "POST /api/convert-video-to-mp3 HTTP/1.1" 200 -


MoviePy - Done.


127.0.0.1 - - [14/Nov/2023 22:14:10] "OPTIONS /api/convert-mp4-to-text HTTP/1.1" 200 -


Proccessing Input........
Converting to text........


127.0.0.1 - - [14/Nov/2023 22:14:36] "POST /api/convert-mp4-to-text HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2023 22:14:37] "GET /api/analyze-audio/why%20I%20was%20elements%20I%20love%20that%20I%20don't%20have%20to%20enemy%20my%20own%20text%20in%20conversations%20in%20After%20Effects%20that's%20a%20lot%20of%20pictures%20unlimited%20download HTTP/1.1" 200 -


Proccessing Input........
Converting to text........


127.0.0.1 - - [14/Nov/2023 22:18:32] "POST /api/convert-mp3-to-text HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2023 22:18:33] "GET /api/analyze-audio/hello%20again%20in%20this%20video%20we%20will%20use%20speech%20recognition%20module%20to%20extract%20text%20from%20a%20voice%20recorded%20in%20a%20MP3%20file%20the%20main%20steps%20for%20these%20are%20first%20we%20need%20to%20install%20speech%20recognition%20module%20then%20if%20you%20have%20a%20Windows%2010%20machine%20with%20Python%203.8.2%20install%20between%20module%20and%20with%20between%20you%20installed%20by%20also%20after%20this%20we%20can%20import%20and%20initialise%20the%20engine%20with%20MP3%20file%20and%20finally%20extract%20and%20print%20the%20text%20first%20we%20need%20to%20install%20speech%20recognition%20so%20install%20speech%20recognition%20it%20enter%20after%20this%20we%20need%20to%20install%20because%20I%20am%20using%20Windows%2010%20with%20Python%203.6%20install%20to%20install%20NTC%20seats%20with%20me%20close%20the%20termi